<a href="https://colab.research.google.com/github/adrielmori/__tempColab__/blob/main/notebooks/Adriel_regioes_sinasc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output
%%capture

!apt install unixodbc-dev --yes
!wget http://download.dremio.com/odbc-driver/1.3.19.1052/dremio-odbc-1.3.19.1052-1.x86_64.rpm 
!apt-get install rpm2cpio
!apt-get install cpio
!rpm2cpio "/content/drive/Shareddrives/MS/utils/dremio-odbc-1.5.0.1001-1.x86_64.rpm"  xzcat | cpio -idmv
!pip install pyodbc

clear_output()

In [ ]:
import pyodbc, pandas as pd

host = "200.137.215.27"
port = 31010
uid = "gabriela"
pwd = "bcR#X*bL9!ul"
driver = "./opt/dremio-odbc/lib64/libdrillodbc_sb64.so"

cnxn = pyodbc.connect("Driver={};ConnectionType=Direct;HOST={};PORT={};AuthenticationType=Plain;UID={};PWD={}".format(driver, host,port,uid,pwd),autocommit=True)

In [ ]:
query = '''SELECT * FROM Dados.sia.pa LIMIT 10'''
dataframe = pd.read_sql(query, cnxn)

dataframe.head()

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,...,PA_ETNIA,uf,competencia,arquivo,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C,PA_INE,PA_NAT_JUR
0,2001586,120000,EP,120040,0000,0000,0000,05,40,I,...,None,AC,08-07,PAAC0807.parquet,None,None,None,None,None,None
1,2001586,120000,EP,120040,0000,0000,0000,05,40,I,...,None,AC,08-07,PAAC0807.parquet,None,None,None,None,None,None
2,2001586,120000,EP,120040,0000,0000,0000,05,40,I,...,None,AC,08-07,PAAC0807.parquet,None,None,None,None,None,None
3,3542734,120000,EP,120040,0000,0000,0000,43,40,M,...,None,AC,08-07,PAAC0807.parquet,None,None,None,None,None,None
4,3542734,120000,EP,120040,0000,0000,0000,43,40,M,...,None,AC,08-07,PAAC0807.parquet,None,None,None,None,None,None


In [ ]:
query = """ 
    SELECT 
        cod_macrorregiao, cod_regsaud, count(*)
    FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
    WHERE cod_macrorregiao=1102
    GROUP BY cod_macrorregiao, cod_regsaud

"""

municipios = pd.read_sql(query, cnxn)
municipios.head()

,cod_macrorregiao,cod_regsaud,EXPR$2
0,1102,11003,4
1,1102,11004,5
2,1102,11001,9


In [ ]:
import os

print(os.getcwd())

/content


In [ ]:
##
# Quantidade de nascidos vivos por região de saúde, agrupando por mês/ano
##
# hie.cod_regsaud,
query = """ 
    SELECT 
        SUBSTRING(sinasc.DTNASC, 3, 8) as DATA, 
        COUNT(*) as NASCIDOS
    FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
    INNER JOIN Dados.sinasc.DN as sinasc
        ON hie.cod_municipio = SUBSTRING(sinasc.CODMUNRES, 1, 6)
    WHERE hie.cod_macrorregiao = 1102
    GROUP BY hie.cod_macrorregiao, SUBSTRING(sinasc.DTNASC, 3, 8) 
    ORDER BY SUBSTRING(DATA, 3, 4), SUBSTRING(DATA, 1, 2) 
"""

sinasc_reg = pd.read_sql(query, cnxn)
# sinasc_reg.to_csv('df_SINASCM_RO_NORTE.csv', index=False)

In [ ]:
query = """
    SELECT cod_macrorregiao, cod_regsaud
    FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
    GROUP BY hie.cod_macrorregiao, cod_regsaud
"""

municipios = pd.read_sql(query, cnxn)
municipios.head()

,cod_macrorregiao,cod_regsaud
0,1102,11003
1,1511,15009
2,1702,17005
3,1702,17008
4,2110,21016


In [ ]:
municipios['cod_macrorregiao'].info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   cod_macrorregiao  116 non-null    object
dtypes: object(1)
memory usage: 1.0+ KB


In [ ]:
len(municipios['cod_macrorregiao'].unique()),municipios['cod_macrorregiao'].unique() 

(116, array(['1102', '1511', '1702', '2110', '2111', '2207', '2209', '2308',
        '2309', '2310', '2402', '2608', '2910', '2911', '2917', '3101',
        '3103', '3105', '3108', '3519', '3527', '3528', '3532', '3533',
        '4105', '4107', '4211', '4212', '4308', '4313', '5104', '1510',
        '2307', '4310', '4311', '2606', '3529', '3530', '4108', '1601',
        '3107', '1512', '1701', '2109', '2502', '2607', '2703', '2704',
        '2801', '2912', '2913', '3111', '3312', '3531', '4106', '5105',
        '5207', '1302', '2501', '2605', '2916', '3526', '4314', '5209',
        '2503', '5007', '5103', '2210', '2914', '3102', '3104', '3110',
        '3112', '3113', '3207', '3208', '3518', '3525', '4210', '4214',
        '4309', '4312', '5005', '5102', '5208', '5210', '3109', '5302',
        '1303', '1201', '2918', '3106', '3520', '5008', '5101', '5206',
        '2208', '1401', '2306', '2401', '3114', '4213', '4215', '1101',
        '1304', '1509', '3205', '3206', '5006', '3524', '29

In [ ]:
macros=municipios['cod_macrorregiao'].unique() 
len(macros)

116

In [ ]:
cnxn = pyodbc.connect("Driver={};ConnectionType=Direct;HOST={};PORT={};AuthenticationType=Plain;UID={};PWD={}".format(driver, host,port,uid,pwd),autocommit=True)

In [ ]:
##
# Quantidade de nascidos vivos por região de saúde, agrupando por mês/ano
##
# hie.cod_regsaud,
query = """ 
    SELECT 
        SUBSTRING(sinasc.DTNASC, 3, 8) as DATA, 
        COUNT(*) as NASCIDOS
    FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
    INNER JOIN Dados.sinasc.DN as sinasc
        ON hie.cod_municipio = SUBSTRING(sinasc.CODMUNRES, 1, 6)
    WHERE hie.cod_macrorregiao = 1102
    GROUP BY hie.cod_macrorregiao, SUBSTRING(sinasc.DTNASC, 3, 8) 
    ORDER BY SUBSTRING(DATA, 3, 4), SUBSTRING(DATA, 1, 2) 
"""

sinasc_reg = pd.read_sql(query, cnxn)
# sinasc_reg.to_csv('df_SINASCM_RO_NORTE.csv', index=False)

In [ ]:
##
# Quantidade de nascidos vivos por região de saúde, agrupando por municipios e mês/ano
##


        #hie.cod_regsaud,
        #hie.cod_municipio,
query = """ 
    SELECT 
        hie.cod_macrorregiao,
         hie.cod_municipio,
        SUBSTRING(sinasc.DTNASC, 3, 8) as DATA, 
        COUNT(*) as NASCIDOS
    FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
    INNER JOIN Dados.sinasc.DN as sinasc
        ON hie.cod_municipio = SUBSTRING(sinasc.CODMUNRES, 1, 6)
    WHERE hie.cod_regsaud = 52001
    GROUP BY hie.cod_regsaud, hie.cod_municipio, SUBSTRING(sinasc.DTNASC, 1, 8) 
    ORDER BY hie.cod_municipio, SUBSTRING(DATA, 3, 4), SUBSTRING(DATA, 1, 2) 
"""

sinasc_mun_reg = pd.read_sql(query, cnxn)
print(sinasc_mun_reg.shape)
sinasc_mun_reg.head()

DatabaseError: ignored

In [ ]:
print(sinasc_mun_reg['cod_municipio'].unique(), len(sinasc_mun_reg['cod_municipio'].unique()))

check = True
for mun in municipios['cod_municipio'].values:
    if mun not in sinasc_mun_reg['cod_municipio'].unique():
        check = False

print(check)

In [ ]:
##
# Quantidade de nascidos vivos por região de saúde, agrupando por mês/ano
##
# hie.cod_regsaud,
query = """ 
    SELECT 
        SUBSTRING(sinasc.DTNASC, 3, 8) as DATA, 
        COUNT(*) as NASCIDOS
    FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
    INNER JOIN Dados.sinasc.DN as sinasc
        ON hie.cod_municipio = SUBSTRING(sinasc.CODMUNRES, 1, 6)
    WHERE hie.cod_regsaud = 52001
    GROUP BY hie.cod_regsaud, SUBSTRING(sinasc.DTNASC, 3, 8) 
    ORDER BY SUBSTRING(DATA, 3, 4), SUBSTRING(DATA, 1, 2) 
"""

sinasc_reg = pd.read_sql(query, cnxn)
sinasc_reg.to_csv('df_SINASCM_REG.csv', index=False)

# Input new data

In [ ]:
query = """
    SELECT cod_macrorregiao, cod_regsaud
    FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
    GROUP BY hie.cod_macrorregiao, cod_regsaud
"""

municipios = pd.read_sql(query, cnxn)
municipios.head()

In [ ]:
macros=municipios['cod_macrorregiao'].unique() 
len(macros)

In [ ]:
len(municipios['cod_macrorregiao'].unique()),municipios['cod_macrorregiao'].unique() 

In [ ]:
regs = pd.read_csv('/content/drive/Shareddrives/MS/case_nascidos/dataset/time_series/regioes_saude/sinasc_regsaude.csv')['cod_regsaud']

In [ ]:
##
# Quantidade de nascidos vivos por região de saúde, agrupando por dia/mês/ano
##

import pandas as pd

for reg in macros:

     
    query = f""" 
        SELECT 
            hie.cod_macrorregiao,
            SUBSTRING(sinasc.DTNASC, 1, 8) as DATA, 
            COUNT(*) as NASCIDOS
        FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
        INNER JOIN Dados.sinasc.DN as sinasc
            ON hie.cod_municipio = SUBSTRING(sinasc.CODMUNRES, 1, 6)
        WHERE hie.cod_macrorregiao = {reg}
        GROUP BY hie.cod_macrorregiao, SUBSTRING(sinasc.DTNASC, 1, 8) 
        ORDER BY SUBSTRING(DATA, 5, 4), SUBSTRING(DATA, 3, 2), SUBSTRING(DATA, 1, 2) 
    """

    sinasc_reg = pd.read_sql(query, cnxn)
    # sinasc_reg

    query_pre = f""" 
        SELECT 
            hie.cod_macrorregiao,
            SUBSTRING(sinasc.DTNASC, 1, 8) as DATA, 
            COUNT(*) as NASCIDOS
        FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
        INNER JOIN Dados.sinasc.preliminar."2021.parquet" as sinasc
            ON hie.cod_municipio = SUBSTRING(sinasc.CODMUNRES, 1, 6)
        WHERE hie.cod_macrorregiao = {reg}
        GROUP BY hie.cod_macrorregiao, SUBSTRING(sinasc.DTNASC, 1, 8) 
        ORDER BY SUBSTRING(DATA, 5, 4), SUBSTRING(DATA, 3, 2), SUBSTRING(DATA, 1, 2) 
    """

    sinasc_reg_pre = pd.read_sql(query_pre, cnxn)
    # sinasc_reg_pre

    query_pre_22 = f""" 
        SELECT 
            hie.cod_macrorregiao,
            SUBSTRING(sinasc.DTNASC, 1, 8) as DATA, 
            COUNT(*) as NASCIDOS
        FROM "Analytics Layer".Territorial."Municípios - Hierarquia Completa" AS hie
        INNER JOIN Dados.sinasc.preliminar."2022.parquet" as sinasc
            ON hie.cod_municipio = SUBSTRING(sinasc.CODMUNRES, 1, 6)
        WHERE hie.cod_macrorregiao = {reg}
        GROUP BY hie.cod_macrorregiao, SUBSTRING(sinasc.DTNASC, 1, 8) 
        ORDER BY SUBSTRING(DATA, 5, 4), SUBSTRING(DATA, 3, 2), SUBSTRING(DATA, 1, 2) 
    """

    sinasc_reg_pre_22 = pd.read_sql(query_pre_22, cnxn)

    df = pd.concat([sinasc_reg, sinasc_reg_pre, sinasc_reg_pre_22], axis=0)
    df.to_csv(f'/content/drive/Shareddrives/MS/case_nascidos/dataset/time_series/macro_regioes_v2/sinasc_macro_{reg}_dia-mes-ano.csv', index=False)
    # break
# df
    # break

In [ ]:
import os

os.mkdir('/content/drive/Shareddrives/MS/case_nascidos/dataset/time_series/regioes_saude/preliminar/2022/')

In [ ]:
!mv /content/drive/Shareddrives/MS/case_nascidos/dataset/time_series/regioes_saude/preliminar/*.csv /content/drive/Shareddrives/MS/case_nascidos/dataset/time_series/regioes_saude/preliminar/2021/

In [ ]:
import pandas as pd

df = pd.concat([sinasc_reg, sinasc_reg_pre], axis=0)
df

,cod_regsaud,DATA,NASCIDOS
0,52001,01012000,52
1,52001,02012000,65
2,52001,03012000,91
3,52001,04012000,63
4,52001,05012000,74
...,...,...,...
360,52001,27122021,75
361,52001,28122021,74
362,52001,29122021,75
363,52001,30122021,76


In [ ]:
query = """
    SELECT DISTINCT 
        SUBSTRING(sinasc.DTNASC, 5, 4) as ANO
    FROM Dados.sinasc.DN AS sinasc
    GROUP BY SUBSTRING(sinasc.DTNASC, 5, 4)
    ORDER BY SUBSTRING(sinasc.DTNASC, 5, 4)
"""

dataframe = pd.read_sql(query, cnxn) 
dataframe['ANO']

0     2000
1     2001
2     2002
3     2003
4     2004
5     2005
6     2006
7     2007
8     2008
9     2009
10    2010
11    2011
12    2012
13    2013
14    2014
15    2015
16    2016
17    2017
18    2018
19    2019
20    2020
Name: ANO, dtype: object